# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    counts = Counter(text)
    vocab = sorted( counts, key=counts.get, reverse=True)
    vocab_to_int = { word: intNum for intNum, word in enumerate(vocab, 1)}
    
    int_to_vocab = { intNum : word for (word, intNum) in vocab_to_int.items() }
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    token_dict = { '.'  : "||PERIOD||",
                   ','  : "||COMMA||",
                   '"'  : "||QUOTE_MARK||",
                   ';'  : "||SEMI_COLON||",
                   '!'  : "||EXCL_MARK||",
                   '?'  : "||QUEST_MARK||",
                   '('  : "||LEFT_PAREN||",
                   ')'  : "||RIGHT_PAREN||",
                   '--' : "||DASH||",
                   '\n' : "||RETURN||"
                  }
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/home/robert/.conda/envs/tf2/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    Input        = tf.placeholder(tf.int32, [None, None], name='input')
    Targets      = tf.placeholder(tf.int32, [None, None], name='targets') 
    LearningRate = tf.placeholder(tf.float32, name = 'learning_rate')
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm_layers = 3
    drop_list = []
    
    for layer in range(lstm_layers):
        lstm_cell = tf.contrib.rnn.BasicLSTMCell( rnn_size, reuse=None )
    #lstm_cell = tf.contrib.rnn.BasicLSTMCell( rnn_size, forget_bias=0.0, state_is_tuple=True, reuse=tf.get_variable_scope().reuse)
        # Dropout assist from Michael Dausmann in the forums
        drop_elem = tf.contrib.rnn.DropoutWrapper( lstm_cell, output_keep_prob=0.75)
        drop_list.append( drop_elem )
    
    Cell = tf.contrib.rnn.MultiRNNCell( drop_list )
    #Cell = tf.contrib.rnn.MultiRNNCell([lstm_cell for _ in range(num_layers)]
    
    InitialState = Cell.zero_state( batch_size, tf.float32 )
    InitialState= tf.identity( InitialState, name='initial_state' )
    
    return Cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size,embed_dim), -1, 1))
    embed_seq = tf.nn.embedding_lookup( embedding, input_data )
    return embed_seq


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    #get_init_cell(batch_size, rnn_size)
   
    Outputs, FinalState = tf.nn.dynamic_rnn( cell, inputs, dtype=tf.float32 )
    FinalState= tf.identity( FinalState, name='final_state' )
    
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    #in_len = len(input_data)
    print("INPUTS:")
    rnn_size
    print("rnn_size = ", rnn_size)
    print("input_data size = ", input_data.get_shape().as_list())
    print( "vocab_size = ", vocab_size )
    print( "embed_dim = ", embed_dim )
    print("------------------------")
    
    #embed_seq = get_embed(input_data, vocab_size, rnn_size)
    embed_seq = get_embed(input_data, vocab_size, embed_dim)
    print("embed_seq size = ", embed_seq.get_shape().as_list())
    
    Outputs, FinalState = build_rnn( cell, embed_seq )
    print("Output size = ", Outputs.get_shape().as_list())
    print("FinalState size = ", FinalState.get_shape().as_list())
        
    # Apply a fully connected layer with a linear activation and vocab_size as the number of outputs.
    #seq_out = tf.concat( Outputs, axis = 1 )
    #x = tf.reshape( seq_out, [ -1, vocab_size ])
 
    Logits = tf.contrib.layers.fully_connected( Outputs, vocab_size, activation_fn = None)
    print("Logits size = ", Logits.get_shape().as_list())
    
    #,in_len = len(x)
    
    #with tf.variable_scope('softmax'):
#        softmax_weight = tf.Variable( tf.truncated_normal(( in_len, vocab_size), stddev=0.1))
#        softmax_bias   = tf.Variable( tf.zeros( vocab_size ))
        
#    Logits = tf.matmul( x, softmax_weight ) + softmax_bias
#    Logits = tf.nn.softmax( Logits, name='sequence_prediction')
    
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

INPUTS:
rnn_size =  256
input_data size =  [128, 5]
vocab_size =  27
embed_dim =  300
------------------------
embed_seq size =  [128, 5, 300]
Output size =  [128, 5, 256]
FinalState size =  [2, 2, None, 256]
Logits size =  [128, 5, 27]
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    with shape (number of batches, 2, batch size, sequence length)
    2 because 1 for inputs, 1 for targets
    """
    # Code adapted from samples in the deep-learning repo and on the Slack forum
    input_len = len( int_text )
    num_batches = np.floor_divide( input_len + 1, batch_size * seq_length  )
    inputs  = np.array( int_text[ : num_batches * batch_size * seq_length ] )
    targets_ = np.array( int_text[ 1 : num_batches * batch_size * seq_length ] )
    targets  = np.hstack(( targets_, inputs[0] ))  # add the wraparound to the beginning element
    
    input_batches  = np.split( inputs.reshape(  batch_size, -1 ), num_batches, 1 )
    target_batches = np.split( targets.reshape( batch_size, -1), num_batches, 1 )
    
    Batches = np.array(list( zip( input_batches, target_batches )))
    
    return Batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 400
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 200
# Embedding Dimension Size
embed_dim = 150
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 25

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)+1
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

INPUTS:
rnn_size =  200
input_data size =  [None, None]
vocab_size =  6780
embed_dim =  150
------------------------
embed_seq size =  [None, None, 150]
Output size =  [None, None, 200]
FinalState size =  [3, 2, None, 200]
Logits size =  [None, None, 6780]


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/107   train_loss = 8.822
Epoch   0 Batch   25/107   train_loss = 6.304
Epoch   0 Batch   50/107   train_loss = 6.373
Epoch   0 Batch   75/107   train_loss = 6.226
Epoch   0 Batch  100/107   train_loss = 6.317
Epoch   1 Batch   18/107   train_loss = 6.147
Epoch   1 Batch   43/107   train_loss = 6.281
Epoch   1 Batch   68/107   train_loss = 5.971
Epoch   1 Batch   93/107   train_loss = 6.091
Epoch   2 Batch   11/107   train_loss = 6.093
Epoch   2 Batch   36/107   train_loss = 6.289
Epoch   2 Batch   61/107   train_loss = 5.868
Epoch   2 Batch   86/107   train_loss = 6.136
Epoch   3 Batch    4/107   train_loss = 6.138
Epoch   3 Batch   29/107   train_loss = 6.084
Epoch   3 Batch   54/107   train_loss = 6.058
Epoch   3 Batch   79/107   train_loss = 6.104
Epoch   3 Batch  104/107   train_loss = 5.951
Epoch   4 Batch   22/107   train_loss = 6.052
Epoch   4 Batch   47/107   train_loss = 5.950
Epoch   4 Batch   72/107   train_loss = 5.827
Epoch   4 Batch   97/107   train_l

Epoch  41 Batch   88/107   train_loss = 3.812
Epoch  42 Batch    6/107   train_loss = 3.856
Epoch  42 Batch   31/107   train_loss = 3.754
Epoch  42 Batch   56/107   train_loss = 3.633
Epoch  42 Batch   81/107   train_loss = 3.769
Epoch  42 Batch  106/107   train_loss = 3.599
Epoch  43 Batch   24/107   train_loss = 3.881
Epoch  43 Batch   49/107   train_loss = 3.778
Epoch  43 Batch   74/107   train_loss = 3.725
Epoch  43 Batch   99/107   train_loss = 3.706
Epoch  44 Batch   17/107   train_loss = 3.767
Epoch  44 Batch   42/107   train_loss = 3.655
Epoch  44 Batch   67/107   train_loss = 3.696
Epoch  44 Batch   92/107   train_loss = 3.710
Epoch  45 Batch   10/107   train_loss = 3.666
Epoch  45 Batch   35/107   train_loss = 3.609
Epoch  45 Batch   60/107   train_loss = 3.514
Epoch  45 Batch   85/107   train_loss = 3.765
Epoch  46 Batch    3/107   train_loss = 3.383
Epoch  46 Batch   28/107   train_loss = 3.518
Epoch  46 Batch   53/107   train_loss = 3.571
Epoch  46 Batch   78/107   train_l

Epoch  83 Batch   69/107   train_loss = 2.663
Epoch  83 Batch   94/107   train_loss = 2.593
Epoch  84 Batch   12/107   train_loss = 2.758
Epoch  84 Batch   37/107   train_loss = 2.693
Epoch  84 Batch   62/107   train_loss = 2.599
Epoch  84 Batch   87/107   train_loss = 2.607
Epoch  85 Batch    5/107   train_loss = 2.661
Epoch  85 Batch   30/107   train_loss = 2.579
Epoch  85 Batch   55/107   train_loss = 2.796
Epoch  85 Batch   80/107   train_loss = 2.683
Epoch  85 Batch  105/107   train_loss = 2.692
Epoch  86 Batch   23/107   train_loss = 2.570
Epoch  86 Batch   48/107   train_loss = 2.631
Epoch  86 Batch   73/107   train_loss = 2.614
Epoch  86 Batch   98/107   train_loss = 2.669
Epoch  87 Batch   16/107   train_loss = 2.594
Epoch  87 Batch   41/107   train_loss = 2.566
Epoch  87 Batch   66/107   train_loss = 2.548
Epoch  87 Batch   91/107   train_loss = 2.627
Epoch  88 Batch    9/107   train_loss = 2.532
Epoch  88 Batch   34/107   train_loss = 2.616
Epoch  88 Batch   59/107   train_l

Epoch 125 Batch   50/107   train_loss = 2.107
Epoch 125 Batch   75/107   train_loss = 2.198
Epoch 125 Batch  100/107   train_loss = 2.049
Epoch 126 Batch   18/107   train_loss = 1.954
Epoch 126 Batch   43/107   train_loss = 1.995
Epoch 126 Batch   68/107   train_loss = 2.098
Epoch 126 Batch   93/107   train_loss = 1.966
Epoch 127 Batch   11/107   train_loss = 2.153
Epoch 127 Batch   36/107   train_loss = 1.939
Epoch 127 Batch   61/107   train_loss = 1.917
Epoch 127 Batch   86/107   train_loss = 2.033
Epoch 128 Batch    4/107   train_loss = 2.024
Epoch 128 Batch   29/107   train_loss = 2.032
Epoch 128 Batch   54/107   train_loss = 2.058
Epoch 128 Batch   79/107   train_loss = 2.041
Epoch 128 Batch  104/107   train_loss = 2.011
Epoch 129 Batch   22/107   train_loss = 2.137
Epoch 129 Batch   47/107   train_loss = 2.081
Epoch 129 Batch   72/107   train_loss = 1.892
Epoch 129 Batch   97/107   train_loss = 2.123
Epoch 130 Batch   15/107   train_loss = 1.992
Epoch 130 Batch   40/107   train_l

Epoch 167 Batch   31/107   train_loss = 1.649
Epoch 167 Batch   56/107   train_loss = 1.697
Epoch 167 Batch   81/107   train_loss = 1.584
Epoch 167 Batch  106/107   train_loss = 1.604
Epoch 168 Batch   24/107   train_loss = 1.697
Epoch 168 Batch   49/107   train_loss = 1.441
Epoch 168 Batch   74/107   train_loss = 1.728
Epoch 168 Batch   99/107   train_loss = 1.547
Epoch 169 Batch   17/107   train_loss = 1.627
Epoch 169 Batch   42/107   train_loss = 1.618
Epoch 169 Batch   67/107   train_loss = 1.724
Epoch 169 Batch   92/107   train_loss = 1.750
Epoch 170 Batch   10/107   train_loss = 1.761
Epoch 170 Batch   35/107   train_loss = 1.704
Epoch 170 Batch   60/107   train_loss = 1.588
Epoch 170 Batch   85/107   train_loss = 1.604
Epoch 171 Batch    3/107   train_loss = 1.614
Epoch 171 Batch   28/107   train_loss = 1.599
Epoch 171 Batch   53/107   train_loss = 1.594
Epoch 171 Batch   78/107   train_loss = 1.596
Epoch 171 Batch  103/107   train_loss = 1.493
Epoch 172 Batch   21/107   train_l

Epoch 209 Batch   12/107   train_loss = 1.351
Epoch 209 Batch   37/107   train_loss = 1.364
Epoch 209 Batch   62/107   train_loss = 1.290
Epoch 209 Batch   87/107   train_loss = 1.330
Epoch 210 Batch    5/107   train_loss = 1.404
Epoch 210 Batch   30/107   train_loss = 1.250
Epoch 210 Batch   55/107   train_loss = 1.465
Epoch 210 Batch   80/107   train_loss = 1.355
Epoch 210 Batch  105/107   train_loss = 1.378
Epoch 211 Batch   23/107   train_loss = 1.329
Epoch 211 Batch   48/107   train_loss = 1.475
Epoch 211 Batch   73/107   train_loss = 1.496
Epoch 211 Batch   98/107   train_loss = 1.400
Epoch 212 Batch   16/107   train_loss = 1.360
Epoch 212 Batch   41/107   train_loss = 1.335
Epoch 212 Batch   66/107   train_loss = 1.370
Epoch 212 Batch   91/107   train_loss = 1.323
Epoch 213 Batch    9/107   train_loss = 1.222
Epoch 213 Batch   34/107   train_loss = 1.345
Epoch 213 Batch   59/107   train_loss = 1.378
Epoch 213 Batch   84/107   train_loss = 1.335
Epoch 214 Batch    2/107   train_l

Epoch 250 Batch  100/107   train_loss = 1.189
Epoch 251 Batch   18/107   train_loss = 1.160
Epoch 251 Batch   43/107   train_loss = 1.046
Epoch 251 Batch   68/107   train_loss = 1.284
Epoch 251 Batch   93/107   train_loss = 1.157
Epoch 252 Batch   11/107   train_loss = 1.114
Epoch 252 Batch   36/107   train_loss = 1.124
Epoch 252 Batch   61/107   train_loss = 1.149
Epoch 252 Batch   86/107   train_loss = 1.176
Epoch 253 Batch    4/107   train_loss = 1.100
Epoch 253 Batch   29/107   train_loss = 1.201
Epoch 253 Batch   54/107   train_loss = 1.224
Epoch 253 Batch   79/107   train_loss = 1.236
Epoch 253 Batch  104/107   train_loss = 1.176
Epoch 254 Batch   22/107   train_loss = 1.301
Epoch 254 Batch   47/107   train_loss = 1.265
Epoch 254 Batch   72/107   train_loss = 1.176
Epoch 254 Batch   97/107   train_loss = 1.301
Epoch 255 Batch   15/107   train_loss = 1.158
Epoch 255 Batch   40/107   train_loss = 1.229
Epoch 255 Batch   65/107   train_loss = 1.309
Epoch 255 Batch   90/107   train_l

Epoch 292 Batch   81/107   train_loss = 0.917
Epoch 292 Batch  106/107   train_loss = 0.995
Epoch 293 Batch   24/107   train_loss = 1.093
Epoch 293 Batch   49/107   train_loss = 0.891
Epoch 293 Batch   74/107   train_loss = 1.149
Epoch 293 Batch   99/107   train_loss = 0.956
Epoch 294 Batch   17/107   train_loss = 1.092
Epoch 294 Batch   42/107   train_loss = 1.023
Epoch 294 Batch   67/107   train_loss = 1.153
Epoch 294 Batch   92/107   train_loss = 1.216
Epoch 295 Batch   10/107   train_loss = 1.144
Epoch 295 Batch   35/107   train_loss = 1.089
Epoch 295 Batch   60/107   train_loss = 1.092
Epoch 295 Batch   85/107   train_loss = 1.015
Epoch 296 Batch    3/107   train_loss = 0.980
Epoch 296 Batch   28/107   train_loss = 0.983
Epoch 296 Batch   53/107   train_loss = 0.926
Epoch 296 Batch   78/107   train_loss = 0.988
Epoch 296 Batch  103/107   train_loss = 0.868
Epoch 297 Batch   21/107   train_loss = 1.005
Epoch 297 Batch   46/107   train_loss = 1.110
Epoch 297 Batch   71/107   train_l

Epoch 334 Batch   62/107   train_loss = 0.849
Epoch 334 Batch   87/107   train_loss = 0.869
Epoch 335 Batch    5/107   train_loss = 0.983
Epoch 335 Batch   30/107   train_loss = 0.841
Epoch 335 Batch   55/107   train_loss = 0.991
Epoch 335 Batch   80/107   train_loss = 0.919
Epoch 335 Batch  105/107   train_loss = 0.870
Epoch 336 Batch   23/107   train_loss = 0.890
Epoch 336 Batch   48/107   train_loss = 0.880
Epoch 336 Batch   73/107   train_loss = 0.935
Epoch 336 Batch   98/107   train_loss = 0.964
Epoch 337 Batch   16/107   train_loss = 0.883
Epoch 337 Batch   41/107   train_loss = 0.986
Epoch 337 Batch   66/107   train_loss = 0.924
Epoch 337 Batch   91/107   train_loss = 0.986
Epoch 338 Batch    9/107   train_loss = 0.809
Epoch 338 Batch   34/107   train_loss = 0.890
Epoch 338 Batch   59/107   train_loss = 0.900
Epoch 338 Batch   84/107   train_loss = 0.893
Epoch 339 Batch    2/107   train_loss = 0.968
Epoch 339 Batch   27/107   train_loss = 1.019
Epoch 339 Batch   52/107   train_l

Epoch 376 Batch   43/107   train_loss = 0.823
Epoch 376 Batch   68/107   train_loss = 0.834
Epoch 376 Batch   93/107   train_loss = 0.831
Epoch 377 Batch   11/107   train_loss = 0.774
Epoch 377 Batch   36/107   train_loss = 0.700
Epoch 377 Batch   61/107   train_loss = 0.861
Epoch 377 Batch   86/107   train_loss = 0.787
Epoch 378 Batch    4/107   train_loss = 0.857
Epoch 378 Batch   29/107   train_loss = 0.822
Epoch 378 Batch   54/107   train_loss = 0.778
Epoch 378 Batch   79/107   train_loss = 0.877
Epoch 378 Batch  104/107   train_loss = 0.923
Epoch 379 Batch   22/107   train_loss = 0.970
Epoch 379 Batch   47/107   train_loss = 0.818
Epoch 379 Batch   72/107   train_loss = 0.790
Epoch 379 Batch   97/107   train_loss = 0.823
Epoch 380 Batch   15/107   train_loss = 0.791
Epoch 380 Batch   40/107   train_loss = 0.792
Epoch 380 Batch   65/107   train_loss = 0.946
Epoch 380 Batch   90/107   train_loss = 0.924
Epoch 381 Batch    8/107   train_loss = 0.860
Epoch 381 Batch   33/107   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor        = loaded_graph.get_tensor_by_name( "input:0" )
    InitialStateTensor = loaded_graph.get_tensor_by_name( "initial_state:0" )
    FinalStateTensor   = loaded_graph.get_tensor_by_name( "final_state:0" )
    ProbsTensor        = loaded_graph.get_tensor_by_name( "probs:0" )
    return ( InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor )


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # Got assistance from the Slack forum here, 
    # especially from Flavio Cordova 
    prob = np.argmax(probabilities)
    next_word = int_to_vocab[prob]
    return next_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [22]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(to barney) linda ronstadt? how'd you get her?
homer_simpson: well, eh, my memory? he's yogurt?
lenny_leonard: wow, moe.(burp)
homer_simpson: oh, well, i get the keys for a new old life is too thing.
homer_simpson: homer, i did this time i think, do about your kid and the best suit on me!
maya:(loud) 'cause no business likes at nuts. / dance!
krusty_the_clown:(impressed) the first prank call. we go can clean.


moe_szyslak: well, what you have on the numbers is, i never turned to be honest of the new glove all the horrible thing really needed.
homer_simpson: well, don't worry? now, here's anything with me and like about sending that five myself?


moe_szyslak: yeah, that's right, homer. it's called the booze with a day and things watch, stupid here.
moe_szyslak: oh, i think i called the cash and built to feed her kick!(small


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.